In [1]:
#https://www.machinelearningplus.com/nlp/topic-modeling-gensim-python/#:~:text=Topic%20Modeling%20is%20a%20technique,in%20the%20Python's%20Gensim%20package.
# Run in python console
import nltk; nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\mmb\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [2]:
#pip install spacy==0.101.0
#https://spacy.io/usage
#pip install pyLDAvis

In [3]:
import re
import numpy as np
import pandas as pd
from pprint import pprint

# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

# spacy for lemmatization
import spacy

# Plotting tools
import pyLDAvis
import pyLDAvis.gensim_models  # don't skip this
import matplotlib.pyplot as plt
%matplotlib inline

# Enable logging for gensim - optional
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)

import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)

C:\Users\mmb\anaconda3\lib\site-packages\sklearn\decomposition\_lda.py:28: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  EPS = np.finfo(np.float).eps


In [4]:
# NLTK Stop words
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
#stop_words.extend(['from', 'subject', 're', 'edu', 'use'])

In [5]:
data =pd.read_csv('tripadvisor_hotel_reviews.csv')
data_text = data[['Review']]
data_text['index'] = data_text.index
reviews = data_text
reviews

,Review,index
0,nice hotel expensive parking got good deal sta...,0
1,ok nothing special charge diamond member hilto...,1
2,nice rooms not 4* experience hotel monaco seat...,2
3,"unique, great stay, wonderful time hotel monac...",3
4,"great stay great stay, went seahawk game aweso...",4
...,...,...
20486,"best kept secret 3rd time staying charm, not 5...",20486
20487,great location price view hotel great quick pl...,20487
20488,"ok just looks nice modern outside, desk staff ...",20488
20489,hotel theft ruined vacation hotel opened sept ...,20489


In [6]:
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations

data_words = list(sent_to_words(reviews.Review))

print(data_words[:1])

[['nice', 'hotel', 'expensive', 'parking', 'got', 'good', 'deal', 'stay', 'hotel', 'anniversary', 'arrived', 'late', 'evening', 'took', 'advice', 'previous', 'reviews', 'did', 'valet', 'parking', 'check', 'quick', 'easy', 'little', 'disappointed', 'non', 'existent', 'view', 'room', 'room', 'clean', 'nice', 'size', 'bed', 'comfortable', 'woke', 'stiff', 'neck', 'high', 'pillows', 'not', 'soundproof', 'like', 'heard', 'music', 'room', 'night', 'morning', 'loud', 'bangs', 'doors', 'opening', 'closing', 'hear', 'people', 'talking', 'hallway', 'maybe', 'just', 'noisy', 'neighbors', 'aveda', 'bath', 'products', 'nice', 'did', 'not', 'goldfish', 'stay', 'nice', 'touch', 'taken', 'advantage', 'staying', 'longer', 'location', 'great', 'walking', 'distance', 'shopping', 'overall', 'nice', 'experience', 'having', 'pay', 'parking', 'night']]


In [7]:
# Build the bigram and trigram models
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[data_words], threshold=100)  

# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

# See trigram example
print(trigram_mod[bigram_mod[data_words[0]]])

['nice', 'hotel', 'expensive', 'parking', 'got', 'good', 'deal', 'stay', 'hotel', 'anniversary', 'arrived', 'late', 'evening', 'took', 'advice', 'previous', 'reviews', 'did', 'valet_parking', 'check', 'quick', 'easy', 'little', 'disappointed', 'non_existent', 'view', 'room', 'room', 'clean', 'nice', 'size', 'bed', 'comfortable', 'woke', 'stiff', 'neck', 'high', 'pillows', 'not', 'soundproof', 'like', 'heard', 'music', 'room', 'night', 'morning', 'loud', 'bangs', 'doors', 'opening_closing', 'hear', 'people', 'talking', 'hallway', 'maybe', 'just', 'noisy', 'neighbors', 'aveda_bath_products', 'nice', 'did', 'not', 'goldfish', 'stay', 'nice', 'touch', 'taken', 'advantage', 'staying', 'longer', 'location', 'great', 'walking_distance', 'shopping', 'overall', 'nice', 'experience', 'having', 'pay', 'parking', 'night']


In [8]:
# Define functions for stopwords, bigrams, trigrams and lemmatization
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

In [9]:
# Remove Stop Words
data_words_nostops = remove_stopwords(data_words)

# Form Bigrams
data_words_bigrams = make_bigrams(data_words_nostops)

# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
# python3 -m spacy download en
nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner'])

# Do lemmatization keeping only noun, adj, vb, adv
data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

print(data_lemmatized[:1])

[['nice', 'hotel', 'expensive', 'parking', 'get', 'good', 'deal', 'stay', 'hotel', 'anniversary', 'arrive', 'late', 'evening', 'take', 'advice', 'previous', 'review', 'valet_parking', 'check', 'quick', 'easy', 'little', 'disappointed', 'non_existent', 'view', 'room', 'room', 'clean', 'nice', 'size', 'bed', 'comfortable', 'wake', 'stiff', 'neck', 'high', 'pillow', 'soundproof', 'hear', 'music', 'room', 'night', 'morning', 'loud', 'bang', 'door', 'opening_close', 'hear', 'people', 'talk', 'hallway', 'maybe', 'noisy', 'neighbor', 'aveda_bath', 'product', 'nice', 'goldfish', 'stay', 'nice', 'touch', 'take', 'advantage', 'stay', 'long', 'location', 'great', 'walking_distance', 'shopping', 'overall', 'nice', 'experience', 'pay', 'parking', 'night']]


In [10]:
data_words_bigrams

[['nice',
  'hotel',
  'expensive',
  'parking',
  'got',
  'good',
  'deal',
  'stay',
  'hotel',
  'anniversary',
  'arrived',
  'late',
  'evening',
  'took',
  'advice',
  'previous',
  'reviews',
  'valet_parking',
  'check',
  'quick',
  'easy',
  'little',
  'disappointed',
  'non_existent',
  'view',
  'room',
  'room',
  'clean',
  'nice',
  'size',
  'bed',
  'comfortable',
  'woke',
  'stiff',
  'neck',
  'high',
  'pillows',
  'soundproof',
  'like',
  'heard',
  'music',
  'room',
  'night',
  'morning',
  'loud',
  'bangs',
  'doors',
  'opening_closing',
  'hear',
  'people',
  'talking',
  'hallway',
  'maybe',
  'noisy',
  'neighbors',
  'aveda_bath',
  'products',
  'nice',
  'goldfish',
  'stay',
  'nice',
  'touch',
  'taken',
  'advantage',
  'staying',
  'longer',
  'location',
  'great',
  'walking_distance',
  'shopping',
  'overall',
  'nice',
  'experience',
  'pay',
  'parking',
  'night'],
 ['ok',
  'nothing',
  'special',
  'charge',
  'diamond',
  'member'

In [11]:
# Create Dictionary
id2word = corpora.Dictionary(data_lemmatized)

# Create Corpus
texts = data_lemmatized

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

#from gensim import corpora, models
#tfidf = models.TfidfModel(bow_corpus)
#corpus_tfidf = tfidf[bow_corpus]

# View
print(corpus[:1])

[[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1), (6, 1), (7, 1), (8, 1), (9, 1), (10, 1), (11, 1), (12, 1), (13, 1), (14, 1), (15, 1), (16, 1), (17, 1), (18, 1), (19, 1), (20, 1), (21, 1), (22, 2), (23, 1), (24, 2), (25, 1), (26, 1), (27, 1), (28, 1), (29, 1), (30, 1), (31, 1), (32, 1), (33, 1), (34, 1), (35, 5), (36, 2), (37, 1), (38, 1), (39, 1), (40, 1), (41, 2), (42, 1), (43, 1), (44, 1), (45, 1), (46, 1), (47, 1), (48, 1), (49, 3), (50, 1), (51, 1), (52, 1), (53, 3), (54, 1), (55, 2), (56, 1), (57, 1), (58, 1), (59, 1), (60, 1), (61, 1)]]


In [12]:
id2word[0]

'advantage'

In [13]:
# Human readable format of corpus (term-frequency)
[[(id2word[id], freq) for id, freq in cp] for cp in corpus[:1]]

[[('advantage', 1),
  ('advice', 1),
  ('anniversary', 1),
  ('arrive', 1),
  ('aveda_bath', 1),
  ('bang', 1),
  ('bed', 1),
  ('check', 1),
  ('clean', 1),
  ('comfortable', 1),
  ('deal', 1),
  ('disappointed', 1),
  ('door', 1),
  ('easy', 1),
  ('evening', 1),
  ('expensive', 1),
  ('experience', 1),
  ('get', 1),
  ('goldfish', 1),
  ('good', 1),
  ('great', 1),
  ('hallway', 1),
  ('hear', 2),
  ('high', 1),
  ('hotel', 2),
  ('late', 1),
  ('little', 1),
  ('location', 1),
  ('long', 1),
  ('loud', 1),
  ('maybe', 1),
  ('morning', 1),
  ('music', 1),
  ('neck', 1),
  ('neighbor', 1),
  ('nice', 5),
  ('night', 2),
  ('noisy', 1),
  ('non_existent', 1),
  ('opening_close', 1),
  ('overall', 1),
  ('parking', 2),
  ('pay', 1),
  ('people', 1),
  ('pillow', 1),
  ('previous', 1),
  ('product', 1),
  ('quick', 1),
  ('review', 1),
  ('room', 3),
  ('shopping', 1),
  ('size', 1),
  ('soundproof', 1),
  ('stay', 3),
  ('stiff', 1),
  ('take', 2),
  ('talk', 1),
  ('touch', 1),
  ('v

In [14]:
#know the number of topics

In [15]:
# Build LDA model
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=20, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)

In [16]:
# Print the Keyword in the 10 topics
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(0,
  '0.038*"say" + 0.034*"check" + 0.027*"ask" + 0.026*"leave" + 0.022*"service" '
  '+ 0.021*"tell" + 0.021*"book" + 0.021*"hour" + 0.020*"desk" + '
  '0.019*"arrive"'),
 (1,
  '0.142*"bottle" + 0.122*"negative" + 0.092*"atmosphere" + 0.079*"fairly" + '
  '0.044*"resturant" + 0.042*"cut" + 0.041*"bonus" + 0.028*"sunset" + '
  '0.026*"spray" + 0.020*"shock"'),
 (2,
  '0.112*"single" + 0.047*"attention" + 0.040*"rough" + 0.038*"rock" + '
  '0.037*"dine" + 0.034*"expedia" + 0.033*"attempt" + 0.028*"necessary" + '
  '0.027*"certain" + 0.027*"excited"'),
 (3,
  '0.139*"spanish" + 0.117*"awesome" + 0.111*"island" + 0.102*"honeymoon" + '
  '0.066*"blue" + 0.045*"funky" + 0.041*"art" + 0.036*"gift" + 0.017*"opening" '
  '+ 0.013*"strawberry"'),
 (4,
  '0.120*"shower" + 0.061*"tv" + 0.051*"light" + 0.039*"bath" + 0.034*"road" + '
  '0.028*"air_conditione" + 0.026*"daughter" + 0.025*"separate" + '
  '0.024*"fridge" + 0.021*"tub"'),
 (5,
  '0.097*"room" + 0.021*"problem" + 0.021*"night" + 0.0

In [17]:
# Compute Perplexity
print('\nPerplexity: ', lda_model.log_perplexity(corpus))  # a measure of how good the model is. lower the better.

# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)


Perplexity:  -12.582820343798762

Coherence Score:  0.31165571063441744


In [18]:
# Visualize the topics
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim_models.prepare(lda_model, corpus, id2word)
vis

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
17    -0.371938 -0.092766       1        1  38.269098
18    -0.373867 -0.080389       2        1  25.961877
5     -0.332470 -0.068888       3        1  14.131318
0     -0.308575  0.005647       4        1  12.368057
4     -0.076517  0.399870       5        1   2.053973
10    -0.026904  0.181577       6        1   1.446039
14     0.043709  0.024916       7        1   0.789477
8      0.053600  0.012868       8        1   0.614463
13     0.066722 -0.001193       9        1   0.612207
16     0.081021 -0.014352      10        1   0.505964
2      0.090290 -0.021660      11        1   0.496892
3      0.086113 -0.018333      12        1   0.482391
6      0.097126 -0.026480      13        1   0.458367
12     0.095838 -0.025353      14        1   0.455536
1      0.098063 -0.026915      15        1   0.436735
19     0.111523 -0.035007      16        1   0.384785
7      0.147538 -0.049664      17        1   0.204457
11     0.161227 -0.052889      18        1   0.164773
15     0.177698 -0.055369      19        1   0.085571
9      0.179802 -0.055620      20        1   0.078019, topic_info=          Term          Freq         Total Category  logprob  loglift
24       hotel  67919.000000  67919.000000  Default  30.0000  30.0000
49        room  60765.000000  60765.000000  Default  29.0000  29.0000
53        stay  33390.000000  33390.000000  Default  28.0000  28.0000
1694    buffet   5323.000000   5323.000000  Default  27.0000  27.0000
169      staff  21232.000000  21232.000000  Default  26.0000  26.0000
...        ...           ...           ...      ...      ...      ...
2042  remember      0.023898      0.992978  Topic20 -11.0188   3.4291
2683   english      0.023898      0.993010  Topic20 -11.0188   3.4290
3551     ocean      0.023898      0.993053  Topic20 -11.0188   3.4290
2830    ground      0.023898      0.993014  Topic20 -11.0188   3.4290
2749     taste      0.023898      0.992964  Topic20 -11.0188   3.4291

[897 rows x 6 columns], token_table=      Topic      Freq        Term
term                             
645       3  0.999755           _
1219     18  0.625263         aaa
1120      7  0.998440    absolute
1321     16  0.999515  absolutely
1654      3  0.999276    actually
...     ...       ...         ...
280       3  0.855835        work
815       4  0.999490       wrong
771       1  0.003922        year
771       2  0.995878        year
618       5  0.999136       young

[646 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[18, 19, 6, 1, 5, 11, 15, 9, 14, 17, 3, 4, 7, 13, 2, 20, 8, 12, 16, 10])

In [19]:
#pip install gensim==3.8.3 --user
import os 
os.environ.update({'MALLET_HOME':r'C:\Users\mmb\Downloads\DIT\Data Mining\Final Project\mallet-2.0.8'})
mallet_path = 'C:\\Users\\mmb\\Downloads\\DIT\\Data Mining\\Final Project\\mallet-2.0.8\\bin\\mallet' # update this path
ldamallet = gensim.models.wrappers.LdaMallet(mallet_path, corpus=corpus, num_topics=20, id2word=id2word)

CalledProcessError: Command 'C:\Users\mmb\Downloads\DIT\Data Mining\Final Project\mallet-2.0.8\bin\mallet import-file --preserve-case --keep-sequence --remove-stopwords --token-regex "\S+" --input C:\Users\mmb\AppData\Local\Temp\7b776e_corpus.txt --output C:\Users\mmb\AppData\Local\Temp\7b776e_corpus.mallet' returned non-zero exit status 1.

In [20]:
import os
os.environ.update({'MALLET_HOME':r'C:/Users/mmb/Desktop/NLP-l1/mallet-2.0.8/'})

mallet_path = 'C:/Users/mmb/Desktop/NLP-l1/mallet-2.0.8/bin/mallet' # update this path

ldamallet = gensim.models.wrappers.LdaMallet(mallet_path, corpus=corpus, num_topics=3, id2word=id2word)

result = (ldamallet.show_topics(num_topics=3, num_words=10,formatted=False))
for each in result:
    print(each)

CalledProcessError: Command 'C:/Users/mmb/Desktop/NLP-l1/mallet-2.0.8/bin/mallet import-file --preserve-case --keep-sequence --remove-stopwords --token-regex "\S+" --input C:\Users\mmb\AppData\Local\Temp\39700d_corpus.txt --output C:\Users\mmb\AppData\Local\Temp\39700d_corpus.mallet' returned non-zero exit status 1.

In [21]:
import os
from gensim.models.wrappers import LdaMallet
os.environ.update({'MALLET_HOME':r'C:/Users/mmb/Destop/NLP-l1/mallet-2.0.8/'})
mallet_path = 'C:\\Users\\mmb\\Desktop\\NLP-l1\\mallet-2.0.8\\bin\\mallet' # update this path
ldamallet = LdaMallet(mallet_path, corpus=corpus, num_topics=20, id2word=id2word)
result=(ldamallet.show_topics(num_topics=20,num_words=5,formatted=False))
for each in result:
    print(each)

CalledProcessError: Command 'C:\Users\mmb\Desktop\NLP-l1\mallet-2.0.8\bin\mallet import-file --preserve-case --keep-sequence --remove-stopwords --token-regex "\S+" --input C:\Users\mmb\AppData\Local\Temp\8c5d5b_corpus.txt --output C:\Users\mmb\AppData\Local\Temp\8c5d5b_corpus.mallet' returned non-zero exit status 1.